In [30]:
import pandas as pd
from sklearn import preprocessing

In [31]:
# Import raw data
data = pd.read_csv("../future_appts_raw.csv")
data

,appointment_id,age_deid,bmi_deid,ethnicity,ethnic_category_black,ethnic_category_unknown,gender_male,imd19_quintile,main_spoken_language_other,current_smoker,...,service_mapping_priority,service_mapping_surgical,weekswaiting,no_show,did_not_attend_risk,did_not_attend_risk_group,prediction_type,case_control,use_case,appt_date
0,14943511,32,27.45,Asian,0,0,1,2,0,0,...,0,0,58.0,NaN,0.287193,High Risk,NaN,NaN,1,2024-05-08
1,15742730,42,27.45,Not Recorded/Stated,0,1,0,1,0,0,...,0,0,5.0,NaN,0.351414,High Risk,NaN,NaN,1,2024-08-02
2,16778162,22,21.70,White,0,0,0,2,0,0,...,0,1,67.0,NaN,0.575675,Very High Risk,NaN,NaN,1,2024-05-20
3,16994098,32,39.45,Not Recorded/Stated,0,1,0,2,0,0,...,0,0,63.0,NaN,0.150000,Moderate Risk,NaN,NaN,1,2024-07-31
4,17047096,62,27.45,Not Recorded/Stated,0,0,1,1,0,0,...,0,0,62.0,NaN,0.340443,High Risk,NaN,NaN,1,2024-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15810,20254158,32,21.70,White,0,0,1,3,0,0,...,0,0,4.0,NaN,0.365332,High Risk,NaN,NaN,1,2024-09-04
15811,20254160,82,27.45,Black,1,0,0,2,0,0,...,0,0,4.0,NaN,0.088879,Low Risk,NaN,NaN,1,2024-09-04
15812,20254163,82,27.45,Black,1,0,0,1,0,1,...,0,0,6.0,NaN,0.102636,Low Risk,NaN,NaN,1,2024-09-04
15813,20283951,32,27.45,Not Recorded/Stated,0,1,1,2,0,0,...,0,0,3.0,NaN,0.111294,Moderate Risk,NaN,NaN,1,2024-07-05


In [32]:
# Check for missing values ie. columns aviailable for analysis
data.isnull().sum()

appointment_id                              0
age_deid                                    0
bmi_deid                                    0
ethnicity                                   0
ethnic_category_black                       0
ethnic_category_unknown                     0
gender_male                                 0
imd19_quintile                              0
main_spoken_language_other                  0
current_smoker                              0
substance_misuse                            0
smoking_unknown                             0
tfc_name                                    0
patient_on_multiple_pathways                0
appt_month                                  0
appt_hour                                   0
appt_duration                               0
hospital_service_display                    3
service_mapping                             0
reason_display                              0
appointment_type                            0
new_appt_flag                     

In [33]:
# Count number of unique appointment IDs
data["appointment_id"].nunique()

15815

In [34]:
# Average did_not_attend risk score
data[(data["did_not_attend_risk_group"]=="High Risk")|(data["did_not_attend_risk_group"]=="Very High Risk")]["did_not_attend_risk"].mean()

0.2993558462111055

### Transform date from object to datetime

In [35]:
data["appt_date"] = pd.to_datetime(data["appt_date"])
data["appt_date"]

0       2024-05-08
1       2024-08-02
2       2024-05-20
3       2024-07-31
4       2024-05-01
           ...    
15810   2024-09-04
15811   2024-09-04
15812   2024-09-04
15813   2024-07-05
15814   2024-10-21
Name: appt_date, Length: 15815, dtype: datetime64[ns]

### Transform categorical data to numeric data

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15815 entries, 0 to 15814
Data columns (total 57 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   appointment_id                        15815 non-null  int64         
 1   age_deid                              15815 non-null  int64         
 2   bmi_deid                              15815 non-null  float64       
 3   ethnicity                             15815 non-null  object        
 4   ethnic_category_black                 15815 non-null  int64         
 5   ethnic_category_unknown               15815 non-null  int64         
 6   gender_male                           15815 non-null  int64         
 7   imd19_quintile                        15815 non-null  int64         
 8   main_spoken_language_other            15815 non-null  int64         
 9   current_smoker                        15815 non-null  int64         
 10

In [37]:
# Encode the ethnicity column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(data["ethnicity"])

# Add the encoded column to the data
data["ethnicity_trans"] = label

data

,appointment_id,age_deid,bmi_deid,ethnicity,ethnic_category_black,ethnic_category_unknown,gender_male,imd19_quintile,main_spoken_language_other,current_smoker,...,service_mapping_surgical,weekswaiting,no_show,did_not_attend_risk,did_not_attend_risk_group,prediction_type,case_control,use_case,appt_date,ethnicity_trans
0,14943511,32,27.45,Asian,0,0,1,2,0,0,...,0,58.0,NaN,0.287193,High Risk,NaN,NaN,1,2024-05-08,0
1,15742730,42,27.45,Not Recorded/Stated,0,1,0,1,0,0,...,0,5.0,NaN,0.351414,High Risk,NaN,NaN,1,2024-08-02,3
2,16778162,22,21.70,White,0,0,0,2,0,0,...,1,67.0,NaN,0.575675,Very High Risk,NaN,NaN,1,2024-05-20,5
3,16994098,32,39.45,Not Recorded/Stated,0,1,0,2,0,0,...,0,63.0,NaN,0.150000,Moderate Risk,NaN,NaN,1,2024-07-31,3
4,17047096,62,27.45,Not Recorded/Stated,0,0,1,1,0,0,...,0,62.0,NaN,0.340443,High Risk,NaN,NaN,1,2024-05-01,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15810,20254158,32,21.70,White,0,0,1,3,0,0,...,0,4.0,NaN,0.365332,High Risk,NaN,NaN,1,2024-09-04,5
15811,20254160,82,27.45,Black,1,0,0,2,0,0,...,0,4.0,NaN,0.088879,Low Risk,NaN,NaN,1,2024-09-04,1
15812,20254163,82,27.45,Black,1,0,0,1,0,1,...,0,6.0,NaN,0.102636,Low Risk,NaN,NaN,1,2024-09-04,1
15813,20283951,32,27.45,Not Recorded/Stated,0,1,1,2,0,0,...,0,3.0,NaN,0.111294,Moderate Risk,NaN,NaN,1,2024-07-05,3


In [38]:
# Encode the tfc_name column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(data["tfc_name"])

# Add the encoded column to the data
data["tfc_name_trans"] = label

In [39]:
# Encode the patient_on_multiple_pathways column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(data["patient_on_multiple_pathways"])

# Add the encoded column to the data
data["patient_on_multiple_pathways_trans"] = label

In [40]:
# Encode the appt_month column

month_mapping ={
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}

# Map the month names to numbers
data["appt_month_trans"] = data["appt_month"].map(month_mapping)

data[['appt_month', 'appt_month_trans']]


,appt_month,appt_month_trans
0,October,10
1,August,8
2,May,5
3,June,6
4,May,5
...,...,...
15810,September,9
15811,September,9
15812,September,9
15813,July,7


In [41]:
# Encode the hospital_service_display column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(data["hospital_service_display"])

# Add the encoded column to the data
data["hospital_service_display_trans"] = label

In [42]:
# Encode the service_mapping column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(data["service_mapping"])

# Add the encoded column to the data
data["service_mapping_trans"] = label

In [43]:
# Encode the reason_display column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(data["reason_display"])

# Add the encoded column to the data
data["reason_display_trans"] = label

In [44]:
# Find out the codes for reason_display that contain 'adult ENT'
print(data[data["reason_display"].str.contains("adult ENT", case=False, na=False)]["reason_display"].unique())
ent_appts = data[data["reason_display"].str.contains("adult ENT", case=False, na=False)]
ent_codes = ent_appts["reason_display_trans"].unique()
ent_codes

['Adult ENT F/Up' 'Adult ENT New' 'Adult ENT Thyroid F/Up'
 'Adult ENT Thyroid New' 'Adult ENT Balance New' 'Adult ENT Results F/Up'
 'Adult ENT Post Op F/Up' 'Adult ENT Voice Clinic New'
 'Adult ENT Voice Clinic F/Up' 'Adult ENT Pre-Assessment F/Up'
 'Adult ENT Nurse Skin Test New' 'Adult ENT Voice Osteopath F/Up'
 'Adult ENT Voice Osteopath New']


array([ 1,  2,  7,  8,  0,  6,  4, 10,  9,  5,  3, 11, 12])

In [45]:
# Encode the appointment_type column

# Create a label encoder object
le = preprocessing.LabelEncoder()

# Fit the label encoder
label = le.fit_transform(data["appointment_type"])

# Add the encoded column to the data
data["appointment_type_trans"] = label

In [46]:
# Encode the did_not_attend_risk_group column

risk_mapping ={
    "Low Risk": 1,
    "Moderate Risk": 2,
    "High Risk": 3,
    "Very High Risk": 4,
}

# Map the month names to numbers
data["did_not_attend_risk_group_trans"] = data["did_not_attend_risk_group"].map(risk_mapping)

data[['did_not_attend_risk_group', 'did_not_attend_risk_group_trans']]

,did_not_attend_risk_group,did_not_attend_risk_group_trans
0,High Risk,3
1,High Risk,3
2,Very High Risk,4
3,Moderate Risk,2
4,High Risk,3
...,...,...
15810,High Risk,3
15811,Low Risk,1
15812,Low Risk,1
15813,Moderate Risk,2


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15815 entries, 0 to 15814
Data columns (total 66 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   appointment_id                        15815 non-null  int64         
 1   age_deid                              15815 non-null  int64         
 2   bmi_deid                              15815 non-null  float64       
 3   ethnicity                             15815 non-null  object        
 4   ethnic_category_black                 15815 non-null  int64         
 5   ethnic_category_unknown               15815 non-null  int64         
 6   gender_male                           15815 non-null  int64         
 7   imd19_quintile                        15815 non-null  int64         
 8   main_spoken_language_other            15815 non-null  int64         
 9   current_smoker                        15815 non-null  int64         
 10

### Export the transformed use case 1 data

In [48]:
# Dropped all transformed columns
data_trans = data.drop(["ethnicity",
"tfc_name",
"patient_on_multiple_pathways",
"appt_month",
"hospital_service_display",
"service_mapping",
#"reason_display",
#"appointment_type",
"did_not_attend_risk_group"], axis =1)

# Export data for use case 1 to CSV
data_trans.to_csv("../future_appts_trans.csv", index=False)

In [49]:
# Filter for adult ENT appointments only
ent_codes = [1,  2,  7,  8,  0,  6,  4, 10,  9,  5,  3, 11, 12]
data_trans_ent = data_trans[data_trans["reason_display_trans"].isin(ent_codes)]

data_trans_ent.to_csv("../future_appts_trans_ENT.csv", index=False)